<a href="https://colab.research.google.com/github/AbhiJeet70/PerceptualLossVAE/blob/main/Perceptual_Loss_in_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'celeba-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F29561%2F37705%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T211638Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D44933434f7f6976a174955c00dc35e1380278abc2ca6e7197e6092fe672ce8a4de6f84d25a8f4367e22d956c6821c8e4405d2ed5ab21f120d0d30b86e4fd8590b31a1f7de480ee757444febea3bf78d3c759017837e1313c7c9969a58678ed153acb3eb52faab2c1555a08169ce2f50725d962fcf9796b4714e25ea5e4e5faba6b05a55f7edae4ac99e84fa0b99b64c97929880ce2b05458fd2344332357751a04c42dd91ab715effcd868cfd6b03508ed989692b155c3f76296a1592d07a38bc9d5e7df95530ffdcbfb999c6fac5e3eab9a03d7feed25b15cd0d4fae7befcd62596302ce83e17b3f1291757daa8064477a891670a7c29b45a82a70bc99a4bd2,celeba-small-images-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2078533%2F3450962%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T211638Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D78725995b0b743cd9b0cbcc6ff9d5f80d9d1cec33aec297800b6f6bf132e3fe937e833192a78a8ebb240ee18565e5ffc990abde7158cd1fb2d5a8ec2ef86256d4b7f07314d70dd51e6091e62f451802ce10bb066210cdfe2c0e859e51ce6a25a82ca1ed806164a799de72a95037c1ef428db6be616036f3a7622ed4464dc8280ec4b3cb5e92e29d27e5535c8de707a99a53e2139daeac8141df2f9c42c109478a64c0ec8ec89185a1780877ad25047a301a3fb86764911ea6500a2514f274a71963a220727b59a05621e361f7d6c28fc5369899bc4d026a76688721f55f907294fc94ac6e3eae3b5ba555ba1f4eb7e4619514fc12e5b99733611efd988c7d632,vgg19-weights:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4827773%2F8160279%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T211638Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D21093dc203458d7d7cfdb439d1a253f612dc7a814b80337d27244a8a76850987787b3838df5af8816bcf1f0eb9f930e8dc87792d43addf755adc80d0d1261cfe75d5b72ba76263273af9d8ac95e19b3b3e7d64c065e1d6cab2e88f32a560eb581b5178759902f2ae0933f39c3e3b0c3476c59e6c527b4f5eb043f14e63cde3f8f514cc7a0d4e85151a8d570ca809a980ef07760eab21b51bb1fb1e365a2eb66dacc33c2b320b0361c63d1e2b941e5d9cb43dd63facfc799eb9d2e04bcdc56da3223f9daa750213451cae704b187cab0c089588302dd7772fbceed17d6d3921f3afe5bcd7aa561eb3989e33070f6ec5a44718cc8eb3596e583d951eba4533c8eb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import os, warnings
import json
import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from keras import backend
from keras import utils
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import CenterCrop
from tensorflow.keras.layers import Resizing
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

# Disable TensorFlow info and warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Constants :
batch_size = 64
img_path = '/kaggle/input/celeba-small-images-dataset/'

In [ ]:
# A short function that converts a file path to an (img, label) pair
# with label is the img itself as our model is an autoencoder.
def process_path(file_path):
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = tf.io.decode_jpeg(img, channels=3)
  #return img, img
  return img

def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.batch(batch_size, drop_remainder=True)
  ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

train_ds = tf.data.Dataset.list_files(img_path + 'training/*.jpg', shuffle=False)
val_ds = tf.data.Dataset.list_files(img_path + 'validation/*.jpg', shuffle=False)
test_ds = tf.data.Dataset.list_files(img_path + 'testing/*.jpg', shuffle=False)

# Use Dataset.map to create a dataset of image, label pairs:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

In [ ]:
#display a few celebrity faces
plt.figure(figsize=(10, 5))
#for image_batch, label_batch in train_ds.take(1):
for image_batch in train_ds.take(1):
    for i in range(10):
        ax = plt.subplot(2, 5, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.axis("off")
plt.show();

In [ ]:
#Build the image preprocesser
def create_preprocesser():

    inputs = keras.Input(shape=(64,64,3))

    preprocessed = Rescaling(1./255)(inputs)

    preprocesser = Model(inputs,preprocessed, name='preprocesser')

    return preprocesser

In [ ]:
#Build the encoder
def create_encoder():

    inputs = keras.Input(shape=(64,64,3))

    # 32x4x4 conv, stride 2 + BN + LeakyReLU
    x = layers.Conv2D(32,(4, 4), strides=2, padding='same', activation=None)(inputs)
    x = layers.BatchNormalization(axis=-1)(x) # axis=-1 as data format is 'channels_last' and we want to normalize each channel
    x = layers.LeakyReLU(alpha=0.3)(x)

    #64x4x4 conv, stride 2 + BN + LeakyReLU
    x = layers.Conv2D(64,(4, 4), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    #128x4x4 conv, stride 2 + BN + LeakyReLU
    x = layers.Conv2D(128,(4, 4), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    #256x4x4 conv, stride 2 + BN + LeakyReLU
    x = layers.Conv2D(256,(4, 4), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # flatten the output
    x = layers.Flatten(data_format='channels_last')(x)

    # distribution parameters
    z_mean = layers.Dense(units=100, activation=None)(x)
    z_log_sigma = layers.Dense(units=100, activation=None)(x)

    # use these parameters to sample new similar points from the latent space:
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = backend.random_normal(shape=(1, 100), mean=0., stddev=0.1)
        #epsilon = backend.random_normal(shape=(1, 100), mean=0., stddev=1.)
        return z_mean + backend.exp(z_log_sigma / 2) * epsilon

    z = layers.Lambda(sampling)([z_mean, z_log_sigma])

    # Create encoder
    encoder = Model(inputs,[z_mean, z_log_sigma, z], name='encoder')
    #encoder.summary()

    return encoder

In [ ]:
#Build the decoder
def create_decoder():

    inputs = keras.Input(shape=(100))

    x = layers.Dense(units=4096, activation='relu')(inputs)
    x = layers.Reshape((4,4,256))(x)

    # upsample + conv128x3x3 + BN + LeakyReLU
    x = layers.UpSampling2D((2,2), data_format='channels_last', interpolation='nearest')(x)
    x = layers.Conv2D(128,(3, 3), padding='same', strides=(1,1))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # upsample + conv64x3x3 + BN + LeakyReLU
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(64,(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # upsample + conv32x3x3 + BN + LeakyReLU
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(32,(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # upsample + conv128x3x3
    x = layers.UpSampling2D((2,2))(x)
    decoded = layers.Conv2D(3,(3, 3), padding='same')(x)

    decoder = Model(inputs, decoded, name='decoder')
    return decoder

In [ ]:
#Build and Train the Plain VAE
class PVAE(keras.Model):

    def __init__(self, preprocesser, encoder, decoder, alpha, beta, **kwargs):
        super(PVAE, self).__init__(**kwargs)
        self.preprocesser = preprocesser
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.loss_wo_weight_tracker = keras.metrics.Mean(name="loss_wo_weight")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.alpha = alpha
        self.beta = beta

    @property
    def metrics(self):
        return [
            self.loss_tracker,
            self.loss_wo_weight_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def forward(self, data):
        # Forward pass
        preprocessed = self.preprocesser(data)
        z_mean, z_log_sigma, z = self.encoder(preprocessed)
        reconstruction = self.decoder(z)
        return z_mean, z_log_sigma,reconstruction

    def compute_loss(self, data, z_mean, z_log_sigma, reconstruction):
        reconstruction_loss = tf.reduce_mean(keras.losses.mean_squared_error(data, reconstruction))
        kl_loss = -0.5 * (1 + z_log_sigma - tf.square(z_mean) - tf.exp(z_log_sigma))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        loss_wo_weight = reconstruction_loss + kl_loss
        loss = self.alpha * reconstruction_loss + self.beta * kl_loss
        return reconstruction_loss, kl_loss, loss, loss_wo_weight

    def train_step(self, data):
        with tf.GradientTape() as tape:
            # Forward pass
            z_mean, z_log_sigma, reconstruction = self.forward(data)
            # Compute the loss value
            reconstruction_loss, kl_loss, loss, loss_wo_weight = self.compute_loss(data, z_mean, z_log_sigma, reconstruction)
        # Compute gradients
        grads = tape.gradient(loss, self.trainable_variables)
        # Update weights
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        # Update metrics (metrics = losses in our case)
        self.loss_tracker.update_state(loss)
        self.loss_wo_weight_tracker.update_state(loss_wo_weight)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        # Return metrics
        return {
            "loss": self.loss_tracker.result(),
            "loss_wo_weight": self.loss_wo_weight_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, data):
        # Forward pass
        z_mean, z_log_sigma, reconstruction = self.forward(data)
        # Compute the loss value
        reconstruction_loss, kl_loss, loss, loss_wo_weight = self.compute_loss(data, z_mean, z_log_sigma, reconstruction)
        # Update metrics (metrics = losses in our case)
        self.loss_tracker.update_state(loss)
        self.loss_wo_weight_tracker.update_state(loss_wo_weight)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        # Return metrics
        return {
            "loss": self.loss_tracker.result(),
            "loss_wo_weight": self.loss_wo_weight_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "reconstruction":reconstruction,
        }

In [ ]:
# summarize history for losses
def display_history(history):

    fig=plt.figure(figsize=(20,5))

    ax = plt.subplot(1, 2, 1)
    plt.plot(history.history['reconstruction_loss'])
    plt.plot(history.history['val_reconstruction_loss'])
    plt.title('reconstruction_loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')

    ax = plt.subplot(1, 2, 2)
    plt.plot(history.history['kl_loss'])
    plt.plot(history.history['val_kl_loss'])
    plt.title('kl_loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')

    plt.show()

In [ ]:
alpha = 1.8
beta = 10
lr = 0.0015

ppreprocesser = create_preprocesser()
pencoder = create_encoder()
pdecoder = create_decoder()
pvae = PVAE(ppreprocesser, pencoder, pdecoder, alpha, beta)
pvae.compile(optimizer=keras.optimizers.Adam(lr))

callbacks = [
    ReduceLROnPlateau(monitor='val_loss_wo_weight', factor=0.2, patience=2, verbose=1, mode='min'),
    ModelCheckpoint(filepath='./pvae/', monitor="val_loss_wo_weight", verbose=1, save_best_only=True)
]
history_pvae = pvae.fit(x=train_ds, epochs=30, validation_data=val_ds, batch_size=batch_size, callbacks=callbacks)

In [ ]:
display_history(history_pvae)

In [ ]:
#Build and Train the VAE with Perceptual Loss
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.vgg19 import preprocess_input

class VAE_123(keras.Model):

    def __init__(self, preprocesser, encoder, decoder, alpha, beta, vgg19_123, **kwargs):
        super(VAE_123, self).__init__(**kwargs)
        self.preprocesser = preprocesser
        self.encoder = encoder
        self.decoder = decoder
        self.alpha = alpha
        self.beta = beta
        self.vgg19_123 = vgg19_123
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.loss_wo_weight_tracker = keras.metrics.Mean(name="loss_wo_weight")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.loss_tracker,
            self.loss_wo_weight_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def call(self, inputs, training=False):
        preprocessed = self.preprocesser(inputs)
        z_mean, z_log_sigma, z = self.encoder(preprocessed)
        reconstruction = self.decoder(z)
        if training:
            return self.compute_loss(inputs, z_mean, z_log_sigma, reconstruction)
        else:
            return reconstruction

    def compute_loss(self, data, z_mean, z_log_sigma, reconstruction):
        data = preprocess_input(data)
        reconstruction = preprocess_input(reconstruction)
        vgg19_data_1, vgg19_data_2, vgg19_data_3 = self.vgg19_123(data)
        vgg19_rec_1, vgg19_rec_2, vgg19_rec_3 = self.vgg19_123(reconstruction)
        reconstruction_loss_1 = tf.reduce_mean(keras.losses.mean_squared_error(vgg19_data_1, vgg19_rec_1))
        reconstruction_loss_2 = tf.reduce_mean(keras.losses.mean_squared_error(vgg19_data_2, vgg19_rec_2))
        reconstruction_loss = (reconstruction_loss_1 + reconstruction_loss_2) / 2
        kl_loss = -0.5 * (1 + z_log_sigma - tf.square(z_mean) - tf.exp(z_log_sigma))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        loss_wo_weight = reconstruction_loss + kl_loss
        loss = self.alpha * reconstruction_loss + self.beta * kl_loss
        return {"loss": loss, "loss_wo_weight": loss_wo_weight, "reconstruction_loss": reconstruction_loss, "kl_loss": kl_loss}

    def train_step(self, data):
        with tf.GradientTape() as tape:
            loss_values = self.call(data, training=True)
            loss = loss_values["loss"]
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss_values

    def test_step(self, data):
        loss_values = self.call(data, training=True)
        return loss_values



In [ ]:
alpha = 1.8
beta = 10
lr = 0.0015

vgg19 = VGG19(include_top=False, weights='/kaggle/input/vgg19-weights/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=(64, 64, 3))
vgg19_123 = Model(inputs=vgg19.input, outputs=[vgg19.get_layer('block1_conv1').output, vgg19.get_layer('block2_conv1').output, vgg19.get_layer('block3_conv1').output])
vgg19_123.trainable = False

preprocesser_123 = create_preprocesser()
encoder_123 = create_encoder()
decoder_123 = create_decoder()
vae_123 = VAE_123(preprocesser_123, encoder_123, decoder_123, alpha, beta, vgg19_123)
vae_123.compile(optimizer=keras.optimizers.Adam(lr), run_eagerly=True)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss_wo_weight', factor=0.2, patience=2, verbose=1, mode='min'),
    ModelCheckpoint(filepath='./vae_123/', monitor="val_loss_wo_weight", verbose=1, save_best_only=True)
]
history_123 = vae_123.fit(x=train_ds, epochs=30, validation_data=val_ds, batch_size=batch_size, callbacks=callbacks)

In [ ]:
display_history(history_123)

In [ ]:
#Reconstruction
test_ds_reconstructed_p = pvae.predict(test_ds)
test_ds_reconstructed_p = test_ds_reconstructed_p['reconstruction']
test_ds_reconstructed_123 = vae_123.predict(test_ds)
test_ds_reconstructed_123 = test_ds_reconstructed_123['reconstruction']

In [ ]:
def display_rec_images(test_ds_reconstructed):

    images = ['182638.jpg','182639.jpg','182640.jpg','182641.jpg','182642.jpg']

    fig=plt.figure(figsize=(20,5))

    for i in range(0,5):

        #display original image
        f = img_path + 'testing/' + images[i]
        img = PIL.Image.open(f)

        ax = plt.subplot(2, 5, i + 1)
        plt.imshow(img)
        plt.axis('off')

        #display reconstructed image
        ax = plt.subplot(2, 5, i + 6)
        plt.imshow(test_ds_reconstructed[i].astype("uint8"))
        plt.axis('off')

    plt.show()

In [ ]:
#Celebrity faces reconstructed using the Plain VAE (top : original images, bottom : reconstructed images)
display_rec_images(test_ds_reconstructed_p)

In [ ]:
#Celebrity faces reconstructed using VAE_123 :
display_rec_images(test_ds_reconstructed_123)

In [ ]:
#Generate new face images
def display_fake_images(decoder):
    z = np.random.normal(loc=0.0, scale=0.5, size=(10,100,1))
    fakefaces = decoder(z)

    plt.figure(figsize=(20, 5))
    for i in range(0, len(fakefaces)):
        ax = plt.subplot(2, 5, i + 1)
        plt.imshow(fakefaces[i].numpy().astype("uint8"))
        plt.axis("off")
    plt.show();

In [ ]:
#from the decoder network of plain variational autoencoder (PVAE) trained with pixel-based loss
display_fake_images(pdecoder)

In [ ]:
#from the decoder network of plain variational autoencoder (VAE_123) trained with perceptual loss
display_fake_images(decoder_123)